<a href="https://colab.research.google.com/github/dangtrantrung/-Ensemble-learning_Bagging_XGBoost_RandomForest/blob/master/train_KeypointsDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dl-course-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F829369%2F1480608%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T082326Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21322021049971b47c53535e99039d98d768612789d06b17ca44637ab9f21697db6746045da716d7cdab5c55c5e2a79af048d5b1fa7872e3cf4eb8aa75734919b6d659205cbb6bfd93c1182a0e255ce8200ff1f4eb236093b5086ce4f1d7b5283dda4faae23f5a1383e0a973e4f662ff8562a907e5f84e31dc8a10cd200b90fcda5b5ef8823c13e627f7368def829115c82175314d4d142026fdb31c06a1b1ea98b38eafd4ee0b6b81ccaf9e6e05adf0326a02af041a558c45f00b54fc4d7288a716d9fce318a398498852253c014080817975cc0f5a8976898e66a3cade20a3b7da334fdec24ed59122e02c08a4f34df82fc24972a4a9c8b738bbd9ff22da54,dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4664047%2F7934410%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T082326Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D48301422978697b807a4a7afad62fe1168adb2d17361b1dcd1db0c127b00879b049f2162d40a819c36d6536f983d6b0af1609c03fa831ecfe42cf1b299c0791ee6b73704ccec9cb1bba7cf06708bc39b3aa1d6677425eae1aa111e617b6a9d1687d75a904a1d32120aa4933c4828ff3d5392a104ed876970c0b3ac713989c69d08f9ce534cfb32ad5ab87576a92f1b5bd2a1ea173f19861ad057e37188e65f6cec86c06516aa6f6db8afc71158c0e5ed4d51b800244cf28719a0237bd603cce0580e4a2a4d7730313f75a2ba6ccadbd76b50a33c4464c847dc6433fbc25a7f3a501a165752bbd41d7991cfa123f9dabe18c325807197f7cc345e30665246926a,model123:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4664120%2F7934517%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T082326Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7860889b0abc6a7a1486978e9512de512fcd233000ddcb2c8fc49019b600235485551ba5092717972eb5d44c8d73379be4d2d74f759ec2261517d08771ff5dee2db8bbe291f447a661c40fa10a5ee006e3ba29215cf9c7fa4b57ba704b07f97c6db62854ebc090f4657d52d8e903bcfe9b46fdea15d3cc3729db9feaaddd5449f74e9a22b426e3f7496055f500809364138e28b5baf6a4ee31d87d5846f3e0b66bc3274a9a512eb925f89e684ee5f7aa8c642a7cbacb513973eff9e2a5cbf8b54c001e7694b1ca5fc5136455f1907d07dda7702e7d5fd97cd3fea3c8e9adaeb6dcaea86f1c691d88ffc42250f6759eef4a7a180ed24773f3efb87b12072c74b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 242157006 bytes downloaded
Downloaded and uncompressed: dl-course-data
[==================================================] 7238138717 bytes downloaded
Downloaded and uncompressed: dataset
[==================================================] 95175350 bytes downloaded
Downloaded and uncompressed: model123
Data source import complete.


<a href="https://colab.research.google.com/github/dangtrantrung/Build-an-AI-ML-Tennis-Analysis-system-with-YOLO-PyTorch-and-Key-Point-Extraction/blob/master/Train_keypoint_model_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8" --header="Cookie: HSID=Ag2OIHvsd2Wub4C7z; SSID=AWnBcQKwDHiTrZAU1; APISID=pltrFZgE9lJ0o1gq/AN9feEHYvs8oHd519; SAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-1PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-3PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; SID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-uttBbVDRolhF-hY16nwHXw0gACgYKAWISAQASFQHGX2MivNTw_E_toJuIRy6LMpKNOBoVAUF8yKpFSmvq7AMjvEWeNc50Zff40076; __Secure-1PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utbSY2jBY1VXuw8gYl5hIO2QACgYKAXsSAQASFQHGX2MihVCJ1PwLozGqZgdSatM9QhoVAUF8yKpgrsTvI8i_UE-YHpoN7Gx-0076; __Secure-3PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utwVfPl2imdPimZJ9tdDZGQAACgYKAUESAQASFQHGX2MiEJ49mV4jME2kttDAV5hwWBoVAUF8yKp80mIgju1lu-q4nI7VsFDM0076; NID=511=efI9IZpxtyJ7Dw1MAUXU8FlzS5jXGewY4Er8HliWc3A0RSWdgvNDyKY66ETjgRyTGWPbWODSmiSeYSBab5SPHVwqbJxd6ZeGW2f6BkHi61UKksXPH0CVJRM1hKpMjHPU5qw7tboM2Mi87NrosV8COB-GCLulLLbjOoSAEQewTe8NVZ5Owq8IkwvxFGfJkmUKEMkFWrw9yb5nTDl3wbZEsGFI92iEdNTSxSRovNCIPN2US-SCFdQ0m2BtvwdiWZbgnn7dSQ8yPA145Kk2BA-ATpJNJ6SJHEHLQY-9CPail9D5qgJgxR925EUg5RGCpEu9wS5xbA62KTa19wAvbAq7Dk3TWc-iX4p1s7ESFyDC7yMpFxiFPJjqkWwFi_ZfiK2TW2t0TQ60DFBxqOytQaLyHrkEvD-CQPVj6OCOP22cZY0Cu61HaAQgFO9pXH-kJUlywzVdbirJumN5gswyaQ49b3KdLcG0Jb7brOMTM24T2nGtQ10hJzsnTwX7dBk3ujqQrI_DGuURvPassPUrIZ0; AEC=Ae3NU9MOEGeKAZjP6INpOYbyMraWAWztmx5pJB_1ILu1furiTy1K37k15u0; __Secure-1PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; __Secure-3PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; 1P_JAR=2024-02-18-08; SIDCC=ABTWhQExCxkfmwCkG1RaEgz8U1ZkPeh3HmLMUdMt8S5cNSsLY5U5rAL6wlvq7dtjRw7zrtAbqsFI; __Secure-1PSIDCC=ABTWhQH0jLeRIS6Tu3LS8DXB5Q3gGDq9LTmlk60FKu795Bf0UbzsOcYWVAE96clq5aAL8i724Q0; __Secure-3PSIDCC=ABTWhQHIFcyv3nZYwp78WXEQal71jCE_ZsGT5lXs8VLr7XDIfFqHcLTIPz4HxzJb9ZnYQ5l2s9eU" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040" -c -O 'tennis_court_det_dataset.zip'

# unzip large dataset

In [3]:
#!unzip dataset.zip

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Create Torch Dataset


In [6]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [7]:
train_dataset = KeypointsDataset("../input/dataset/data/images","../input/dataset/data/data_train.json")
val_dataset = KeypointsDataset("../input/dataset/data/images","../input/dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [8]:
len(train_dataset)

6630

In [9]:
len(val_dataset)

2211

In [10]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    modelx = checkpoint['model']
    modelx.load_state_dict(checkpoint['state_dict'])
    for parameter in modelx.parameters():
        parameter.requires_grad = False

    modelx.eval()

    return modelx

In [11]:
# Option 1: passing weights param as string
#model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")
filepath='/kaggle/input/model123/resnet50-0676ba61.pth'
model = models.resnet50()
model.load_state_dict(torch.load(filepath))

<All keys matched successfully>

In [12]:
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [13]:
model = model.to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [14]:
import numpy

In [ ]:
epochs=20
for epoch in range(epochs):
    total_correct = 0
    total_samples = 0
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        predicted = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, kps)
#         # Update the running total of correct predictions and samples
#         correct_predictions = sum(predicted == kps[0:]).item()
#         total_correct += correct_predictions
#         total_samples += kps.size(0)
#         # Calculate the accuracy for this epoch
#         accuracy = 100 * total_correct / total_samples
#  print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}%')
        loss.backward()

        optimizer.step()


        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")



Epoch 0, iter 0, loss: 14452.123046875
Epoch 0, iter 10, loss: 14748.5390625
Epoch 0, iter 20, loss: 15438.3076171875
Epoch 0, iter 30, loss: 14621.904296875
Epoch 0, iter 40, loss: 13971.884765625
Epoch 0, iter 50, loss: 13360.0810546875
Epoch 0, iter 60, loss: 12999.853515625
Epoch 0, iter 70, loss: 11946.5419921875
Epoch 0, iter 80, loss: 11657.8330078125
Epoch 0, iter 90, loss: 11536.166015625
Epoch 0, iter 100, loss: 10808.859375
Epoch 0, iter 110, loss: 11435.7607421875
Epoch 0, iter 120, loss: 10577.14453125
Epoch 0, iter 130, loss: 10597.7001953125
Epoch 0, iter 140, loss: 8608.333984375
Epoch 0, iter 150, loss: 9222.1552734375
Epoch 0, iter 160, loss: 8492.0966796875


In [ ]:
torch.save(model, "keypoints_model.pth")